# Dataset and Dataloader

In [3]:
import numpy as np
import pandas as pd
import torch 
import torch.autograd as autograd 
import torch.nn as nn 
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import *

In [2]:
class CancerDataset(Dataset):
    
    # constructor
    def __init__(self, df_X, df_y):
        assert len(df_X) == len(df_y)
        
        self.data_list = torch.tensor(df_X.values)
        self.target_list = torch.tensor(df_y.values)
        
        assert(len(self.data_list) == len(self.target_list))
      
    # return the length of dataset
    def __len__(self):
        return len(self.data_list)
    
    # return the key-th element of dataset
    def __getitem__(self, key):
        
        return self.data_list[key], self.target_list[key]
    

In [5]:
df = pd.read_csv('train_ml2_2021.csv')
X = df.drop(columns=['target', 'problem_id'])
y = df.target

In [6]:
X_train, X_val, y_train, y_val = train_test_split(X, y, train_size=0.8, random_state=1)

In [7]:
train_dataset = CancerDataset(X_train, y_train)
valid_dataset = CancerDataset(X_val, y_val)

batch_size=100
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=True)